## RTA Monitor

This notebook will help you record, and live plot the temperature of the RTA over time. 



Define the *maximum* number of seconds you would like to monitor the process 
> Note that once all heating steps are complete, no more temperature data can be read. The reading process may terminate before the actual specified number of seconds to read for.

In [1]:
# remeber, this number is in seconds!
# The actual reading time may 
READ_FOR = 15*60 

Where should I save plot data?

In [2]:
from pathlib import Path 

data_path = Path(r"RTA-data\Sept 22 Manab Hallbar")
data_path.mkdir(parents=True, exist_ok=True)

Great. Here are some useful `python` libraries that we will be using. Please help me import them by clicking the run button next to the cell.

In [3]:
import h5py
import numpy as np
import plotly.graph_objs as go
import matplotlib.pyplot as plt 

from SerialReader import SerialReaderPlotter

s = SerialReaderPlotter("COM8", baudrate=19200, timeout=60)

ModuleNotFoundError: No module named 'serial'

## Run
Execute the following cell to start reading from the RTA.

In [27]:
fig = go.FigureWidget(data=[])
fig.update_layout(
    template="plotly_white",
    xaxis_title="Time (s)",
    yaxis_title="Temperature (C)",
)

"""
You the convenience of allowing to do other things 
while the serial reader is reading when "s.writer.join()" is commented out
"""
s.start_reading(READ_FOR)
# fig.update_layout(xaxis_range=[0,READ_FOR])
s.start_writing(fig)
# s.writer.join()

(Reader   ) Starting
(Plotter  ) Starting


## Display the Live Plot

In [28]:
fig

FigureWidget({
    'data': [{'line': {'width': 1},
              'type': 'scatter',
              'uid': 'f494…

## Saving Data

Write down any short comments about the process 

In [29]:
COMMENT = "450C_Sample_5mbar_70ramp_actual"

In [30]:
from datetime import datetime
date = datetime.now().strftime("%Y_%m_%d-%I-%M_%p")
print("Today: {}".format(date))

Today: 2021_09_23-09-14_PM


In [31]:
for i, scatter in enumerate(fig.data):
    filename = data_path/Path("RTA_Run_{}_".format(i) + COMMENT + "_{}.h5".format(date))
    hf = h5py.File(filename, 'w')
    hf.create_dataset('time_arr', data=np.array(scatter.x[1:], dtype=float)) 
    hf.create_dataset('temperature_arr', data=np.array(scatter.y[1:], dtype=float))
    hf.close()

(Reader   ) Exiting


## Loading Data

In [14]:
# Specify which directory the data will be read from
read_path = Path(r"RTA-data\Sept 22 Manab Hallbar")

In [15]:
previous_runs = {f.name: h5py.File(f, "r") for f in read_path.glob("*.h5")}

In [16]:
read = go.FigureWidget(data=[])
read.update_layout(
    template="plotly_white",
    xaxis_title="Time (s)",
    yaxis_title="Temperature (C)",
)

for k in previous_runs.keys():
    read.add_scatter(x=[], 
                    y=[], 
                    line=dict(width=1))

    read_dat = read.data[-1]
    
    x = previous_runs[k]["time_arr"][:]
    y = previous_runs[k]["temperature_arr"][:]
    with read.batch_update():
        read_dat.x += tuple(x)
        read_dat.y += tuple(y) 
    previous_runs[k].close()
read